### Problema
**El Triángulo**  
Javier estaba un día practicando con su instrumento favorito: el triángulo. El triángulo es un instrumento musical tan espectacular que sus notas se escriben como números enteros. Este día Javier se propuso componer una canción de una forma bastante peculiar. Tomó $n$ enteros (notas) aleatorios y los escribió en una lista $a$. Una melodía válida es una subsecuencia de $a$ en la que todos sus números adyacentes cumplen que:
* Se diferencian en 1.
* Son congruentes módulo 7.

La canción de Javier debe contener exactamente 4 melodías que cumplan con lo anterior y además no intercepten entre sí. Ayude a Javier encontrando una canción que maximice las notas usadas.


Consideraciones
* Se asume que la secuencia de notas tiene longitud mayor o igual que 4.
* La solución implementada devuelve la cantidad máxima de notas en la canción.

### Modelación y Correctitud
Con el objetivo de maximizar las subsecuencias que se pueden extraer de $a$ sin que se intercepten, se debe tener en cuenta crear un grafo dirigido $G$ que respete este criterio, por lo que todos los valores $a_i$ son vértices de $G$ ya que cada uno por si solo es una subsecuencia de $a$.
Luego cada uno de estos vértices podría formar una subsecuencia con otros vértices cuyo valor este en una posición superior en el array y cumpla los requisitos expuestos en el problema, finalmente:
* $G$ grafo del sistema
* $\forall i,j ~\in~ a$: $~~~i \rightarrow j \in A(G)~~$ si $~~i < j~~$ y se cumple que: $~|a_i - a_j| = 1~~$ o $~~(a_i - a_j)\%7 = 0$
* $cap(u,v)$ función de capacidad en la arista
* $cost(u,v)$ se agrega la función del costo para tomar en cuenta esta arista en la secuencia

Para la resolución de este problema se realizará un algoritmo de flujo máximo (en este caso 4) con costo mínimo.  
Redefinamos el grafo original de la siguiente forma:
* $G_f = <V,A>$ una red de flujo
* $s$: fuente $\in V(G_f)$
* $t$: receptor $\in V(G_f)~$ al cual solo llega flujo máximo 4, que son las 4 subsecuencias que se desea encontrar
* $\forall v_i \in V(G)$ se divide en $v_{i1} \in G_f$ y $v_{i2} \in G_f$ y se adiciona una arista dirigida $v_{i1} \rightarrow v_{i2} \in A(G_f)$ donde $~cap(v_{i1},v_{i2}) = 1~$ y $~cost(v_{i1},v_{i2}) = -1~$ que representa la decisión de tomar este elemento como parte de la subsecuencia a formar
* $\forall v_i \in V(G)~$ se tiene $~s \rightarrow v_{i1} \in A(G_f)~$ ya que, como se mencionó anteriormente, cada elemento por si solo puede ser una subsecuencia o el punto de partida de una, donde $~cap(s,v_{i1}) = 1~$ y $~cost(s,v_{i1}) = 0$
* $\forall i,j~~$ si $v_i \rightarrow v_j \in A(G)$ entonces $~v_{i2} \rightarrow v_{j1} \in A(G_f)~$ con $~cap(v_{i2},v_{j1}) = 1~$ y $~cost(v_{i2},v_{j1}) = 0$
* $\forall v_i \in V(G)~$ se tiene $~v_{i2} \rightarrow t \in A(G_f)~$ para terminar la subsecuencia, donde $~cap(v_{i2},t) = 1~$ y $~cost(v_{i2},t) = 0$

Luego, en la red residual para cada cada arista de retroceso de las aristas $~u \rightarrow v$ se cumple que $cap_{0}(v,u) = 0 $ y $cost(v,u) =  ~-cost(u,v)$ 

La idea tras la modelación anterior está asociada, a que cada vez que se utiliza una nota en un camino aumentativo se disminuye el costo de dicho camino, por lo que el algoritmo trataría en todo momento de maximizar el flujo a 4 con el menor costo posible (es decir con la mayor cantidad de notas). 

La diferencia de este algoritmo con el algoritmo de flujo máximo clásico, es que cuando se computa un camino aumentativo, en lugar de buscar el camino mas corto hacia el destino (usando BFS), se busca el camino de costo mínimo hacia el destino (usando el algoritmo de Bellman-Ford en este caso, dado que existen aristas con costo negativo).

El grafo original es un DAG, dado que solo se tienen aristas en una sola dirección (hacia delante) en la lista de notas. Luego en la red residual se forman ciclos con la introducción de las aristas de retroceso, pero el costo del ciclo formado por una arista del grafo original y su arista de retroceso es cero ($cost(v,u) =  ~-cost(u,v)$). Por tanto, es posible utilizar Bellman-Ford para encontrar un camino de costo mínimo en la red residual, dado que se garantiza que no existen ciclos de costo negativo en dicha red.


### Solución
Una posible implementación del algoritmo es la siguiente:

Para la creación del grafo y partiendo de las aclaraciones realizadas en la sección anterior, se recibe una lista de enteros que representa las notas, por cuestión de comodidad y sabiendo que las aristas solo conectarán a un $a_i$ con un $a_j$ donde $i < j$, es fácil notar que si se tiene un array (o en este caso, un diccionario) con todas las posiciones de los vértices, se podría distinguir entre una arista del grafo y una de retroceso, por los valores de sus posiciones.
Construimos el grafo de la red residual con una iteración lineal inversa: conectamos los vértices duplicados, la fuente y el receptor con los vértices adecuados entre ambos (creando también, ambos tipos de aristas) y por cada iteración, se conectan con la posición de las notas que tienen valor $+1$, $-1$, $\%7$ a dicho valor que se han ido guardando a lo largo de la iteración (que tendrán posición superior).  
En el peor de los casos cada nota podría relacionarse con todos sus superiores, este algoritmo tiene un costo de $O(n * (n-1)) \rightarrow O(n^2-n)$ donde $n = |a|$.
 
Notar que la cantidad de aristas de la red residual resultante, es $O(n^2)$

In [1]:
from collections import defaultdict as dd
from math import inf
class edge:
    def __init__(self, cap:int, cost:int):
        self.cap = cap
        self.cost = cost

def add_edge(graph:dict[int, dict[int, edge]], v:int, u:int, cap:int, cost:int):
    graph[v][u] = edge(cap,  cost) 
    graph[u][v] = edge(  0, -cost)

def add_edge_list(graph:dict[int, dict[int, edge]], v:int, index:list[int]):
    for elem in index: add_edge(graph, v, elem, 1, 0)

def create_residual_graph(a:list[int]):
    graph:dict[int, dict[int, edge]] = dd(lambda:dd(lambda:None)) 
    n = len(a)
    s = 0
    t = 2*n+1
    d = 2*n+2
    refer:dict[int, list[int]] = dd(list)          
    modul:dict[int, list[int]] = dd(list)        

    for i in range(n, 0, -1):
        value = a[i-1]
        add_edge(graph, s, 2*i-1, 1, 0)       
        add_edge(graph, 2*i-1, 2*i, 1, -1)    
        add_edge(graph, 2*i, t, 1, 0)     
          
        ig = refer[value+1]
        il = refer[value-1]
        im = modul[value%7]
        
        if len(ig) > 0: add_edge_list(graph, 2*i, ig)
        if len(il) > 0: add_edge_list(graph, 2*i, il)
        if len(im) > 0: add_edge_list(graph, 2*i, im)
        refer[value].append(2*i-1)
        modul[value%7].append(2*i-1)     
    add_edge(graph, t, d, 4, 0)

    return graph, s, d

Algoritmo Bellman-Ford

La complejidad computacional de este algoritmo es $O(|V||E|)$, en este caso la cantidad de aristas en el grafo residual es $O(|V|^2)$ y la cantidad de vértices es $O(n)$ (donde n es la longitud de la secuencia de entrada), luego la complejidad es $O(n^{3})$.

In [2]:
def Bellman_Ford(G:dict[int, dict[int, edge]], s:int):
    d = dd(lambda:inf)
    d[s] = 0
    pi = dd(lambda:None)
    for _ in range(len(G.keys())-1):
        for v1,edges in G.items():
            for v2,p in edges.items():
                if not p==None and not p.cap == 0 and not d[v1]==inf and d[v2]> d[v1]+ p.cost:
                    d[v2]=d[v1]+p.cost
                    pi[v2]=v1
    for v1,edges in G.items():
        for v2,p in edges.items():
            if not p==None and not p.cap == 0 and not d[v2]==inf and not d[v1]==inf and  d[v2]>d[v1]+p.cost:
                return None
    return pi

Algoritmo MinCost-MaxFlow


El costo de esta implementación es igual la cantidad de veces que se ejecuta el ciclo while multiplicado por el costo del algoritmo find_path, adicionado a C(create_residual_graph)+ $O(n^{2})$, donde $n$ es la cantidad de vértices del grafo original.

En cada iteración del ciclo while el flujo que se pasa por la red original aumenta en la capacidad del camino aumentativo en cuestión, que es como mínimo 1. Luego la cantidad máxima de veces que se ejecuta este ciclo es igual al valor del flujo máximo, en este caso un valor constante 4.

El algoritmo find_path computa el camino de costo mínimo hasta el destino en la red residual (O($n^{3}$)) y luego reconstruye dicho camino ($O(n^{2})$), por lo cual el costo es O($n^{3}$), donde $n$ es el tamaño de la secuencia de entrada.

Finalmente, y teniendo en cuenta que C(create_residual_graph)=$O(n^{2})$, la complejidad del algoritmo es O($n^{3}$)

In [3]:
def min_cost_flow(a:list[int],residual_graph):
    G_f, s, t = residual_graph(a)
    output = 0
    flow = dd(lambda:dd(lambda:None))

    for v1,edges in G_f.items():
        for v2,_ in edges.items():
            if v2 > v1: flow[v1][v2]=0
    
    path, cap = find_path(G_f, s, t)   
    while not len(path)==0:
        for v1, v2 in path:
            if  v2 < v1: 
                if G_f[v1][v2].cost == -1: output-=1
                flow[v2][v1] -= cap  
                G_f[v1][v2].cap = G_f[v1][v2].cap - cap   
                G_f[v2][v1].cap = G_f[v2][v1].cap + cap   
            else:                   
                if G_f[v1][v2].cost == -1: output+=1
                flow[v1][v2] += cap                   
                G_f[v1][v2].cap = G_f[v1][v2].cap - cap    
                G_f[v2][v1].cap = G_f[v2][v1].cap + cap     
        path, cap = find_path(G_f, s, t)   
    return flow, output

def find_path(G:dict[int, dict[int, edge]], s:int, t:int):
    pi = Bellman_Ford(G, s)
    cap = inf
    path = []
    if not pi[t]==None:
        current = t
        while not current == s:
            previous = pi[current]
            cap = min(G[previous][current].cap,cap)
            path.append((previous,current))
            current=previous
    return path,cap

### Probemos algunos ejemplos:

In [4]:
a = [1,3,5,4,4,7,9,11]
result,max = min_cost_flow(a,create_residual_graph)
print(max)
a=[1]*100
result,max = min_cost_flow(a,create_residual_graph)
print(max)

7
100


### Mejoras
Una posible forma de mejorar la complejidad del algoritmo es disminuir el orden de la cantidad de aristas del grafo original. Para ello hay un grupo de cuestiones a tener en cuenta: 

Sean:
* $n_{i}$ la nota que corresponde a la posición $i$ de la secuencia de entrada 
* $n_{j}~|~ n_{j}=n_{i}+1 \wedge \forall n_{k}~ (k>i \wedge n_{k}=n_{i}+1 \Rightarrow k>j)$
* $n_{l}~|~ n_{l}=n_{i}-1 \wedge \forall n_{k}~ (k>i \wedge n_{k}=n_{i}-1 \Rightarrow k>l)$
* $n_{p}~|~ n_{p}\equiv n_{i}(7) \wedge \forall n_{k}~ (k>i \wedge n_{k}\equiv n_{i}(7)\Rightarrow k>p)$


Entonces  $\forall n_{k}|~ |n_{k}-n_{i}|=0  \vee n_{k}\equiv n_{i}(7) $ se cumple que:

Si $|n_{k}-n_{i}|=0$ entonces $n_{k}=n_{i}+1 \vee n_{k}=n_{i}-1  $:

* $n_{k}=n_{i}+1 \Rightarrow n_{k}=n_{j}$
* $n_{k}=n_{i}-1 \Rightarrow n_{k}=n_{l}$


Si $n_{k}\equiv n_{i}(7)$ entonces $n_{k}\equiv n_{p}(7)$

Luego si desde cada nota $n_{i}$ tenemos una arista hacia cada una de las notas $n_{j}$, $n_{l}$, $n_{p}$ y desde cada una de ellas tenemos aristas hacia cada una de las notas del mismo valor o que dejan el mismo resto al dividirlas entre 7, logramos que cada una de las notas alcanzables desde $n_{i}$ sean alcanzables desde alguna de las notas $n_{j}$, $n_{l}$ o $n_{p}$. 

Por tanto en el grafo por cada nota $n_{i}$ tendremos 4 vértices:

* $v_{i,1}$: vértice al que llegarán todas las aristas provenientes de vértices  $v_{j,1}$ tales que $n_{j}$ coincide con $n_{i}$  y las aristas provenientes de vértices $v_{j,4}$ tales que la diferencia modular entre $n_{j}$ y $n_{i}$ es 1.
 Desde este vérice se tiene una arista hacia el siguiente vértice $v_{j,1}$, tal que $n_{i} = n_{j}$

* $v_{i,2}$: vértice al que llegarán todas las aristas provenientes de vértices  $v_{j,2}$ cuyo valor  $n_{j}$ es congruente módulo 7 con $n_{i}$.
 Desde este vérice se tiene una arista hacia el siguiente vértice $v_{j,2}$, tal que $n_{i} \equiv n_{j} (7)$

* $v_{i,3}$ vértice al que están conectados los dos anteriores, mediante una arista de costo 0 y capacidad 1, cada uno respectivamente. El vértice auxiliar usado como origen, también tiene una arista de costo 0 y capacidad 1 hacia este vértice.

* $v_{i,4}$ vértice al que está conectado $v_{3}$, mediante una arista con costo -1 y capacidad 1, que análogamente a la primera modelación vista, representa la inclusión de esta nota en la canción.
Desde este vérice se tienen aristas hacia los próximos vértices $v_{j,1}$, 
$v_{k,1}$ y $v_{l,2}$ tales que $n_{i}= n_{j}-1$, $n_{i}= n_{k}+1$ y $n_{i} \equiv n_{l} (7)$ respectivamente. También se incluye la arista hacia el vértice auxiliar de salida.

Nota: Todas las aristas para las cuales no se especificó costo ni capacidad tienen valores de 0 y 1 respectivamente.

 Luego, por cada una de las notas, se tienen a lo sumo 6 aristas que salen de alguno de los vértices especificados anteriormente, de manera que la cantidad de aristas del grafo resultante es $O(n)$, donde n es la cantidad de notas en la secuencia  de entrada.




### Complejidad 

La reducción del orden de la cantidad de aristas en el grafo (ahora $O(n)$) permite que el algoritmo mejorado tenga complejidad $O(n^2)$, dado que ahora el algoritmo Bellman-Ford tiene complejidad $O(n^2)$.

### Implementación
Para obtener una implementación con esta mejora solo necesitamos cambiar el método residual_graph que se usa en el algoritmo min_cost_flow, por otro que logre modelar el grafo como se describe anteriormente: 

In [5]:
def create_reduce_residual_graph(a:list[int]):
    graph:dict[int, dict[int, edge]] = dd(lambda:dd(lambda:None))  
    n = len(a)
    s = 0
    t = 4*n+1
    d = 4*n+2
    refer:dict[int, int] = {}          
    modul:dict[int, int] = {}         

    for i in range(n, 0, -1):
        value = a[i-1]
        add_edge(graph, s, 4*i-1, 1, 0)              
        add_edge(graph, 4*i-3, 4*i-1, 1, 0)          
        add_edge(graph, 4*i-2, 4*i-1, 1, 0)
        add_edge(graph, 4*i-1, 4*i, 1, -1) 
        add_edge(graph, 4*i, t, 1, 0)            
        
        iequal_valid = igreater_valid = ilower_valid = imodule_valid = False  
        try:
            ie = refer[value]
            iequal_valid = True
        except: None
        try:
            ig = refer[value+1]
            igreater_valid = True
        except: None
        try:
            il = refer[value-1]
            ilower_valid = True
        except: None
        try:
            im = modul[value%7]
            imodule_valid = True
        except: None
        
        if iequal_valid: add_edge(graph, 4*i-2, 4*ie-2, 4, 0)
        if igreater_valid and (not iequal_valid or ig < ie): add_edge(graph, 4*i, 4*ig-2, 1, 0)
        if ilower_valid   and (not iequal_valid or il < ie): add_edge(graph, 4*i, 4*il-2, 1, 0)
        if imodule_valid: 
            add_edge(graph, 4*i-3, 4*im-3, 4, 0)
            add_edge(graph, 4*i, 4*im-3, 1, 0)
        refer[value] = modul[value%7] = i
    add_edge(graph, t, d, 4, 0)
    return graph, s, d

### Probemos algunos ejemplos:

In [6]:
a = [1,3,5,4,4,7,9,11]
result,max = min_cost_flow(a,create_reduce_residual_graph)
print(max)
a=[1]*100
result,max = min_cost_flow(a,create_reduce_residual_graph)
print(max)
a=[1]*1000
result,max = min_cost_flow(a,create_reduce_residual_graph)
print(max)

7
100
1000
